# PC Algorithm based MonitorRank

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from collections import defaultdict, OrderedDict

import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["font.size"] = 7
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['axes.linewidth'] = 1.0
plt.rcParams['axes.grid'] = True

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore', FutureWarning)

import sys
sys.path.append("../")

from notebooklib.save import run_tsdr, save_tsdr, load_tsdr, load_tsdr_by_chaos

In [3]:
DATASET_ID = "9n6mf"

In [4]:
from meltria.priorknowledge.priorknowledge import SockShopKnowledge

pk = SockShopKnowledge(
    target_metric_types={
        "containers": True,
        "services": True,
        "middlewares": True,
        "nodes": False,
    },
    mappings={"nodes-containers": {}},
)

In [5]:
import functools
import multiprocessing

from diagnoser.diag import prepare_init_graph, fix_edge_directions_in_causal_graph, find_connected_subgraphs
from eval.groundtruth import check_cause_metrics
import diagnoser.metric_node as mn

# https://github.com/joblib/joblib/pull/366#issuecomment-267603530
def with_timeout(timeout):
    def decorator(decorated):
        @functools.wraps(decorated)
        def inner(*args, **kwargs):
            pool = multiprocessing.pool.ThreadPool(1)
            async_result = pool.apply_async(decorated, args, kwargs)
            try:
                return async_result.get(timeout)
            except multiprocessing.TimeoutError:
                record = args[1]
                print(f"Timeouted {record.chaos_case_full()}")
                pool.terminate()
                return
        return inner
    return decorator

# @with_timeout(10*60)  # 10 min timeout

## Overall evaluation

In [6]:
datasets = load_tsdr_by_chaos(
    DATASET_ID, suffix="hdbscan_sbd_only_ctnrs", revert_normalized_time_series=True,
)

In [7]:
from diagnoser import diag
from notebooklib import rank

In [16]:
import joblib

@with_timeout(10*60)  # 10 min timeout
def diagnose_and_rank(reduced_df, record, **kwargs):
    G, ranks = diag.build_and_walk_causal_graph(
        reduced_df, record.pk,
        pc_library="cdt",
        pc_citest_alpha=0.05,
        pc_citest="fisher-z",
        pc_variant="stable",
        disable_orientation=False,
        walk_method="monitorrank",
        root_metric_type="latency",
    )
    if len(ranks) == 0:
        print(f"Failed to diagnose {record.chaos_case_full()} with {len(ranks)} ranks")
        return None
    return G, rank.create_rank_as_dataframe(ranks, DATASET_ID, record)

ranked_datasets = []
for (_, _), somethings_records in datasets.items():
    records = []
    for record, data_df_by_metric_type in somethings_records:
        reduced_df = pd.concat([
            data_df_by_metric_type["services"][-1],
            data_df_by_metric_type["containers"][-1],
            data_df_by_metric_type["middlewares"][-1],
        ], axis=1)
        records.append((record, reduced_df))

    results = joblib.Parallel(n_jobs=4, backend="multiprocessing")(
        joblib.delayed(diagnose_and_rank)(reduced_df, record)
        for record, reduced_df in records
    )
    assert results is not None
    ranked_datasets.extend(result for result in results if result is not None)
ranked_datasets = [_[1] for _ in ranked_datasets]

Failed to diagnose user/pod-network-latency/3 with 0 ranks


In [48]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    ranks_df = rank.create_rank_as_dataframe_for_multiple_cases_from_frames(ranked_datasets)
    display(ranks_df)

metric_name  \
dataset_id target_app chaos_type          chaos_comp   chaos_idx                                                   
9n6mf      sock-shop  pod-network-latency user-db      4                     c-carts-db_cpu_system_seconds_total   
                                                       4                      c-carts-db_cpu_usage_seconds_total   
                                                       4                     c-carts-db_blkio_device_usage_total   
                                                       4                                   c-carts-db_memory_rss   
                                                       4                                     c-front-end_sockets   
                                                       4                                      s-carts_throughput   
                                                       4                                  c-front-end_memory_rss   
                                                       4                        c-carts-db_fs_writes_bytes_total   
                                                       4                     c-carts-db_memory_working_set_bytes   
                                                       4                         c-carts_cpu_usage_seconds_total   
                                                       3                                       s-user_throughput   
                                                       3                                   s-shipping_throughput   
                                                       3                                     s-catalogue_latency   
                                                       3                                     s-orders_throughput   
                                                       3                                        s-orders_latency   
                                                       3                                      s-front-end_errors   
                                                       3                            c-catalogue_file_descriptors   
                                                       3                    c-catalogue_memory_working_set_bytes   
                                                       3                                     s-front-end_latency   
                                                       2              c-carts-db_cpu_cfs_throttled_periods_total   
                                                       2                                  s-catalogue_throughput   
                                                       2                                      s-carts_throughput   
                                                       2                               c-carts-db_fs_reads_total   
                                                       2                                     s-front-end_latency   
                                                       1                                          s-user_latency   
                                                       1                       c-shipping_cpu_user_seconds_total   
                                                       1                                   s-shipping_throughput   
                                                       1                                         s-carts_latency   
                                                       1                                      c-shipping_sockets   
                                                       1              c-shipping_cpu_cfs_throttled_periods_total   
                                                       1                                      c-shipping_threads   
                                                       1                                        s-orders_latency   
                                                       1                                     s-orders_throughput   
                                         

In [63]:
rank.create_localization_score_as_dataframe(ranked_datasets, pk, k=10, group_by_cause_type=True)

no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, payment
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user-db
no cause metrics

#cases (metric)  AC@K (metric)  \
9n6mf sock-shop pod-cpu-hog         1                14       0.571429   
                                    2                14       0.857143   
                                    3                14       0.928571   
                                    4                14       0.928571   
                                    5                14       1.000000   
                                    6                14       1.000000   
                                    7                14       1.000000   
                                    8                14       1.000000   
                                    9                14       1.000000   
                                    10               14       1.000000   
                pod-memory-hog      1                11       0.181818   
                                    2                11       0.181818   
                                    3                11       0.454545   
                                    4                11       0.545455   
                                    5                11       0.909091   
                                    6                11       0.909091   
                                    7                11       0.909091   
                                    8                11       0.909091   
                                    9                11       0.909091   
                                    10               11       1.000000   
                pod-network-latency 1                 2       0.500000   
                                    2                 2       1.000000   
                                    3                 2       1.000000   
                                    4                 2       1.000000   
                                    5                 2       1.000000   
                                    6                 2       1.000000   
                                    7                 2       1.000000   
                                    8                 2       1.000000   
                                    9                 2       1.000000   
                                    10                2       1.000000   

                                        AVG@K (metric)  #cases (container)  \
9n6mf sock-shop pod-cpu-hog         1         0.571429                  32   
                                    2         0.714286                  32   
                                    3         0.785714                  32   
                                    4         0.821429                  32   
                                    5         0.857143                  32   
                                    6         0.880952                  32   
                                    7         0.897959                  32   
                                    8         0.910714                  32   
                                    9         0.920635                  32   
                                    10        0.928571                  32   
                pod-memory-hog      1         0.181818                  42   
                                    2         0.181818                  42   
                                    3         0.272727                  42   
                                    4         0.340909                  42   
                                    5         0.454545                  42   
                                    6         0.530303                  42   
                                    7         0.584416                  42   
                                    8         0.625000                  42   
                                    9         0.656566                  42   
                                    10        0.690909                  42   
                pod-network-latency 1         0.500000                  35   
                             

In [67]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(rank.create_localization_score_as_dataframe(ranked_datasets, pk, k=10, group_by_cause_type=False, group_by_cause_comp=True))

no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts-db
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts-db
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts-db
no cause metrics: 9n6m

#cases (metric)  AC@K (metric)  \
9n6mf sock-shop carts        1                 2       0.500000   
                             2                 2       0.500000   
                             3                 2       1.000000   
                             4                 2       1.000000   
                             5                 2       1.000000   
                             6                 2       1.000000   
                             7                 2       1.000000   
                             8                 2       1.000000   
                             9                 2       1.000000   
                             10                2       1.000000   
                carts-db     1                 4       0.750000   
                             2                 4       0.750000   
                             3                 4       0.750000   
                             4                 4       0.750000   
                             5                 4       1.000000   
                             6                 4       1.000000   
                             7                 4       1.000000   
                             8                 4       1.000000   
                             9                 4       1.000000   
                             10                4       1.000000   
                catalogue    1                 4       0.500000   
                             2                 4       0.500000   
                             3                 4       0.500000   
                             4                 4       0.500000   
                             5                 4       1.000000   
                             6                 4       1.000000   
                             7                 4       1.000000   
                             8                 4       1.000000   
                             9                 4       1.000000   
                             10                4       1.000000   
                catalogue-db 1                 1       1.000000   
                             2                 1       1.000000   
                             3                 1       1.000000   
                             4                 1       1.000000   
                             5                 1       1.000000   
                             6                 1       1.000000   
                             7                 1       1.000000   
                             8                 1       1.000000   
                             9                 1       1.000000   
                             10                1       1.000000   
                orders       1                 4       0.000000   
                             2                 4       0.750000   
                             3                 4       0.750000   
                             4                 4       0.750000   
                             5                 4       0.750000   
                             6                 4       0.750000   
                             7                 4       0.750000   
                             8                 4       0.750000   
                             9                 4       0.750000   
                             10                4       1.000000   
                orders-db    1                 2       0.000000   
                             2                 2       0.500000   
                             3                 2       1.000000   
                             4                 2       1.000000   
                             5                 2       1.000000   
                             6                 2       1.000000   
                             7                 2       1.000000   
                             8                 2       1.000000   
                             9                 2       1.000000   
             

In [69]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(rank.create_localization_score_as_dataframe(ranked_datasets, pk, k=10, group_by_cause_type=True, group_by_cause_comp=True))

no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts-db
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts-db
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts-db
no cause metrics: 9n6m

#cases (metric)  \
9n6mf sock-shop carts        pod-cpu-hog         1                 0   
                                                 2                 0   
                                                 3                 0   
                                                 4                 0   
                                                 5                 0   
                                                 6                 0   
                                                 7                 0   
                                                 8                 0   
                                                 9                 0   
                                                 10                0   
                             pod-memory-hog      1                 2   
                                                 2                 2   
                                                 3                 2   
                                                 4                 2   
                                                 5                 2   
                                                 6                 2   
                                                 7                 2   
                                                 8                 2   
                                                 9                 2   
                                                 10                2   
                             pod-network-latency 1                 0   
                                                 2                 0   
                                                 3                 0   
                                                 4                 0   
                                                 5                 0   
                                                 6                 0   
                                                 7                 0   
                                                 8                 0   
                                                 9                 0   
                                                 10                0   
                carts-db     pod-cpu-hog         1                 3   
                                                 2                 3   
                                                 3                 3   
                                                 4                 3   
                                                 5                 3   
                                                 6                 3   
                                                 7                 3   
                                                 8                 3   
                                                 9                 3   
                                                 10                3   
                             pod-memory-hog      1                 1   
                                                 2                 1   
                                                 3                 1   
                                                 4                 1   
                                                 5                 1   
                                                 6                 1   
                                                 7                 1   
                                                 8                 1   
                                                 9                 1   
                                                 10                1   
                             pod-network-latency 1                 0   
                                                 2                 0   
                                                 3                 0   
                                                 4                 0   
                                                 5                 0   
                     